**Секція 1. Логістична регресія з нуля.**

Будемо крок за кроком будувати модель лог регресії з нуля для передбачення, чи буде врожай більше за 80 яблук (задача подібна до лекційної, але на класифікацію).

Давайте нагадаємо основні формули для логістичної регресії.

### Функція гіпотези - обчислення передбачення у логістичній регресії:

$$
\hat{y} = \sigma(x W^T + b) = \frac{1}{1 + e^{-(x W^T + b)}}
$$

Де:
- $ \hat{y} $ — це ймовірність "позитивного" класу.
- $ x $ — це вектор (або матриця для набору прикладів) вхідних даних.
- $ W $ — це вектор (або матриця) вагових коефіцієнтів моделі.
- $ b $ — це зміщення (bias).
- $ \sigma(z) $ — це сигмоїдна функція активації.

### Як обчислюється сигмоїдна функція:

Сигмоїдна функція $ \sigma(z) $ має вигляд:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Ця функція перетворює будь-яке дійсне значення $ z $ в інтервал від 0 до 1, що дозволяє інтерпретувати вихід як ймовірність для логістичної регресії.

### Формула функції втрат для логістичної регресії (бінарна крос-ентропія):

Функція втрат крос-ентропії оцінює, наскільки добре модель передбачає класи, порівнюючи передбачені ймовірності $ \hat{y} $ із справжніми мітками $ y $. Формула наступна:

$$
L(y, \hat{y}) = - \left[ y \cdot \log(\hat{y}) + (1 - y) \cdot \log(1 - \hat{y}) \right]
$$

Де:
- $ y $ — це справжнє значення (мітка класу, 0 або 1).
- $ \hat{y} $ — це передбачене значення (ймовірність).



1.
Тут вже наведений код для ініціювання набору даних в форматі numpy. Перетворіть `inputs`, `targets` на `torch` тензори. Виведіть результат на екран.

In [1]:
import torch
import numpy as np

In [2]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

inputs, targets

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[0.],
         [1.],
         [1.],
         [0.],
         [1.]]))

2. Ініціюйте ваги `w`, `b` для моделі логістичної регресії потрібної форми зважаючи на розмірності даних випадковими значеннями з нормального розподілу. Лишаю тут код для фіксації `random_seed`.

In [3]:
torch.random.manual_seed(1)

In [4]:
w = torch.randn(1, 3, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(w)
print(b)

tensor([[0.6614, 0.2669, 0.0617]], requires_grad=True)
tensor([0.6213], requires_grad=True)


3. Напишіть функцію `model`, яка буде обчислювати функцію гіпотези в логістичній регресії і дозволяти робити передбачення на основі введеного рядка даних і коефіцієнтів в змінних `w`, `b`.

  **Важливий момент**, що функція `model` робить обчислення на `torch.tensors`, тож для математичних обчислень використовуємо фукнціонал `torch`, наприклад:
  - обчсилення $e^x$: `torch.exp(x)`
  - обчсилення $log(x)$: `torch.log(x)`
  - обчислення середнього значення вектору `x`: `torch.mean(x)`

  Використайте функцію `model` для обчислення передбачень з поточними значеннями `w`, `b`.Виведіть результат обчислень на екран.

  Проаналізуйте передбачення. Чи не викликають вони у вас підозр? І якщо викликають, то чим це може бути зумовлено?

In [5]:
def model(x, w, b):
    q = x @ w.t() + b
    print(q)
    return 1 / (1 + torch.exp(-q))

In [6]:
preds = model(inputs, w, b)
preds

tensor([[69.4361],
        [88.2410],
        [97.5041],
        [81.8390],
        [76.1967]], grad_fn=<AddBackward0>)


tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], grad_fn=<MulBackward0>)

The perdiction has all positive values, it seems to be connected to how the reandm weights were generated

4. Напишіть функцію `binary_cross_entropy`, яка приймає на вхід передбачення моделі `predicted_probs` та справжні мітки в даних `true_labels` і обчислює значення втрат (loss)  за формулою бінарної крос-ентропії для кожного екземпляра та вертає середні втрати по всьому набору даних.
  Використайте функцію `binary_cross_entropy` для обчислення втрат для поточних передбачень моделі.

In [7]:
def binary_cross_entropy(predicted_probs, true_labels):
    predicted_probs = torch.clamp(predicted_probs, min=1e-5, max=1 - 1e-5)
    loss = - (true_labels * torch.log(predicted_probs) + (1 - true_labels) * torch.log(1 - predicted_probs))

    return torch.mean(loss)


In [8]:
loss = binary_cross_entropy(preds, targets)
loss

tensor(4.6046, grad_fn=<MeanBackward0>)

i had errors with log(0) so i had to cap the probas at really close to 0 and 1 instead of 0 and 1

5. Зробіть зворотнє поширення помилки і виведіть градієнти за параметрами `w`, `b`. Проаналізуйте їх значення. Як гадаєте, чому вони саме такі?

In [9]:
loss.backward()

print(w)
print(w.grad)
print(b)
print(b.grad)

tensor([[0.6614, 0.2669, 0.0617]], requires_grad=True)
tensor([[0., 0., 0.]])
tensor([0.6213], requires_grad=True)
tensor([0.])


Not sure why but i get either 0s or nans as gradients.

**Що сталось?**

В цій задачі, коли ми ініціювали значення випадковими значеннями з нормального розподілу, насправді ці значення не були дуже гарними стартовими значеннями і привели до того, що градієнти стали дуже малими або навіть рівними нулю (це призводить до того, що градієнти "зникають"), і відповідно при оновленні ваг у нас не буде нічого змінюватись. Це називається `gradient vanishing`. Це відбувається через **насичення сигмоїдної функції активації.**

У нашій задачі ми використовуємо сигмоїдну функцію активації, яка має такий вигляд:

   $$
   \sigma(z) = \frac{1}{1 + e^{-z}}
   $$


Коли значення $z$ дуже велике або дуже мале, сигмоїдна функція починає "насичуватись". Це означає, що для великих позитивних $z$ сигмоїда наближається до 1, а для великих негативних — до 0. В цих діапазонах градієнти починають стрімко зменшуватись і наближаються до нуля (бо градієнт - це похідна, похідна на проміжку функції, де вона паралельна осі ОХ, дорівнює 0), що робить оновлення ваг неможливим.

![](https://editor.analyticsvidhya.com/uploads/27889vaegp.png)

У логістичній регресії $ z = x \cdot w + b $. Якщо ваги $w, b$ - великі, значення $z$ також буде великим, і сигмоїда перейде в насичену область, де градієнти дуже малі.

Саме це сталося в нашій задачі, де великі випадкові значення ваг викликали насичення сигмоїдної функції. Це в свою чергу призводить до того, що під час зворотного поширення помилки (backpropagation) модель оновлює ваги дуже повільно або зовсім не оновлює. Це називається проблемою **зникнення градієнтів** (gradient vanishing problem).

**Що ж робити?**
Ініціювати ваги маленькими значеннями навколо нуля. Наприклад ми можемо просто в існуючій ініціалізації ваги розділити на 1000. Можна також використати інший спосіб ініціалізації вагів - інформація про це [тут](https://www.geeksforgeeks.org/initialize-weights-in-pytorch/).

Як це робити - показую нижче. **Виконайте код та знову обчисліть передбачення, лосс і виведіть градієнти.**

А я пишу пояснення, чому просто не зробити

```
w = torch.randn(1, 3, requires_grad=True)/1000
b = torch.randn(1, requires_grad=True)/1000
```

Нам потрібно, аби тензори вагів були листовими (leaf tensors).

1. **Що таке листовий тензор**
Листовий тензор — це тензор, який був створений користувачем безпосередньо і з якого починається обчислювальний граф. Якщо такий тензор має `requires_grad=True`, PyTorch буде відслідковувати всі операції, виконані над ним, щоб правильно обчислювати градієнти під час навчання.

2. **Чому ми використовуємо `w.data` замість звичайних операцій**
Якщо ми просто виконали б операції, такі як `(w - 0.5) / 100`, ми б отримали **новий тензор**, який вже не був би листовим тензором, оскільки ці операції створюють **новий** тензор, а не модифікують існуючий.

  Проте, щоб залишити наші тензори ваги `w` та зміщення `b` листовими і продовжити можливість відстеження градієнтів під час тренування, ми використовуємо атрибут `.data`. Цей атрибут дозволяє **виконувати операції in-place (прямо на існуючому тензорі)** без зміни самого об'єкта тензора. Отже, тензор залишається листовим, і PyTorch може коректно обчислювати його градієнти.

3. **Чому важливо залишити тензор листовим**
Якщо тензор більше не є листовим (наприклад, через проведення операцій, що створюють нові тензори), ви не зможете отримати градієнти за допомогою `w.grad` чи `b.grad` після виклику `loss.backward()`. Це може призвести до втрати можливості оновлення параметрів під час тренування моделі. В нашому випадку ми хочемо, щоб тензори `w` та `b` накопичували градієнти, тому вони повинні залишатись листовими.

**Висновок:**
Ми використовуємо `.data`, щоб виконати операції зміни значень на ваги і зміщення **in-place**, залишаючи їх листовими тензорами, які можуть накопичувати градієнти під час навчання. Це дозволяє коректно працювати механізму зворотного поширення помилки (backpropagation) і оновлювати ваги моделі.

5. Виконайте код та знову обчисліть передбачення, лосс і знайдіть градієнти та виведіть всі ці тензори на екран.

In [10]:
torch.random.manual_seed(1)
w = torch.randn(1, 3, requires_grad=True)  # Листовий тензор
b = torch.randn(1, requires_grad=True)     # Листовий тензор

# in-place операції
w.data = w.data / 1000
b.data = b.data / 1000

In [11]:
preds = model(inputs, w, b)
print(preds)

loss = binary_cross_entropy(preds, targets)
print(loss)

loss.backward()

print(w)
print(w.grad)
print(b)
print(b.grad)

tensor([[0.0694],
        [0.0882],
        [0.0975],
        [0.0818],
        [0.0762]], grad_fn=<AddBackward0>)
tensor([[0.5174],
        [0.5220],
        [0.5244],
        [0.5204],
        [0.5190]], grad_fn=<MulBackward0>)
tensor(0.6829, grad_fn=<MeanBackward0>)
tensor([[6.6135e-04, 2.6692e-04, 6.1677e-05]], requires_grad=True)
tensor([[ -5.4417, -18.9853, -10.0682]])
tensor([0.0006], requires_grad=True)
tensor([-0.0794])


6. Напишіть алгоритм градієнтного спуску, який буде навчати модель з використанням написаних раніше функцій і виконуючи оновлення ваг. Алгоритм має включати наступні кроки:

  1. Генерація прогнозів
  2. Обчислення втрат
  3. Обчислення градієнтів (gradients) loss-фукнції відносно ваг і зсувів
  4. Налаштування ваг шляхом віднімання невеликої величини, пропорційної градієнту (`learning_rate` домножений на градієнт)
  5. Скидання градієнтів на нуль

Виконайте градієнтний спуск протягом 1000 епох, обчисліть фінальні передбачення і проаналізуйте, чи вони точні?

In [12]:
def gradient_descent(model, inputs, targets, learning_rate, epochs):
  w = torch.randn(1, 3, requires_grad=True)
  b = torch.randn(1, requires_grad=True)

  w.data = w.data / 1000
  b.data = b.data / 1000

  for epoch in range(epochs):
    loss = binary_cross_entropy(model(inputs, w, b), targets)
    loss.backward()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    with torch.no_grad():
      w.data -= learning_rate * w.grad
      b.data -= learning_rate * b.grad
      w.grad.zero_()
      b.grad.zero_()

  return model(inputs, w, b)

final_preds = gradient_descent(model, inputs, targets, 0.001, 1000)

Показано результат, скорочений до останніх рядків (5000).
        [-7.1031],
        [ 5.2806]], grad_fn=<AddBackward0>)
Epoch 168/1000, Loss: 0.1723877191543579
tensor([[-0.4168],
        [ 0.9076],
        [ 4.6801],
        [-7.1203],
        [ 5.2933]], grad_fn=<AddBackward0>)
Epoch 169/1000, Loss: 0.1720665544271469
tensor([[-0.4193],
        [ 0.9093],
        [ 4.6839],
        [-7.1374],
        [ 5.3061]], grad_fn=<AddBackward0>)
Epoch 170/1000, Loss: 0.17174668610095978
tensor([[-0.4218],
        [ 0.9111],
        [ 4.6878],
        [-7.1544],
        [ 5.3188]], grad_fn=<AddBackward0>)
Epoch 171/1000, Loss: 0.17142809927463531
tensor([[-0.4242],
        [ 0.9128],
        [ 4.6916],
        [-7.1715],
        [ 5.3315]], grad_fn=<AddBackward0>)
Epoch 172/1000, Loss: 0.1711108386516571
tensor([[-0.4267],
        [ 0.9146],
        [ 4.6955],
        [-7.1884],
        [ 5.3442]], grad_fn=<AddBackward0>)
Epoch 173/1000, Loss: 0.17079493403434753
tensor([[-0.4291],
        [ 0

In [13]:
final_preds = final_preds > 0.5
final_preds = final_preds.float()

print(final_preds)
print(targets)

tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.]])
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.]])


**Секція 2. Створення лог регресії з використанням функціоналу `torch.nn`.**

Давайте повторно реалізуємо ту ж модель, використовуючи деякі вбудовані функції та класи з PyTorch.

Даних у нас буде побільше - тож, визначаємо нові масиви.

In [14]:
# Вхідні дані (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

# Таргети (apples > 80)
targets = np.array([[0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1],
                    [0],
                    [1],
                    [1],
                    [0],
                    [1]], dtype='float32')

In [15]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

final_preds = gradient_descent(model, inputs, targets, 0.001, 1000)

final_preds = final_preds > 0.5
final_preds = final_preds.float()

print(final_preds)
print(targets)

Показано результат, скорочений до останніх рядків (5000).
        [ -1.2795],
        [  1.6443],
        [  5.7380],
        [-13.0625],
        [  9.9436],
        [ -1.2795],
        [  1.6443],
        [  5.7380],
        [-13.0625],
        [  9.9436]], grad_fn=<AddBackward0>)
Epoch 691/1000, Loss: 0.08506198227405548
tensor([[ -1.2807],
        [  1.6454],
        [  5.7393],
        [-13.0707],
        [  9.9501],
        [ -1.2807],
        [  1.6454],
        [  5.7393],
        [-13.0707],
        [  9.9501],
        [ -1.2807],
        [  1.6454],
        [  5.7393],
        [-13.0707],
        [  9.9501]], grad_fn=<AddBackward0>)
Epoch 692/1000, Loss: 0.08497422188520432
tensor([[ -1.2819],
        [  1.6465],
        [  5.7405],
        [-13.0788],
        [  9.9566],
        [ -1.2819],
        [  1.6465],
        [  5.7405],
        [-13.0788],
        [  9.9566],
        [ -1.2819],
        [  1.6465],
        [  5.7405],
        [-13.0788],
        [  9.9566]], grad_fn

7. Завантажте вхідні дані та мітки в PyTorch тензори та з них створіть датасет, який поєднує вхідні дані з мітками, використовуючи клас `TensorDataset`. Виведіть перші 3 елементи в датасеті.



In [45]:
from torch.utils.data import TensorDataset, DataLoader

In [46]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[0.],
         [1.],
         [1.]]))

8. Визначте data loader з класом **DataLoader** для підготовленого датасету `train_ds`, встановіть розмір батчу на 5 та увімкніть перемішування даних для ефективного навчання моделі. Виведіть перший елемент в дата лоадері.

In [47]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [102.,  43.,  37.],
         [ 91.,  88.,  64.],
         [ 69.,  96.,  70.]]),
 tensor([[0.],
         [1.],
         [0.],
         [1.],
         [1.]])]

9. Створіть клас `LogReg` для логістичної регресії, наслідуючи модуль `torch.nn.Module` за прикладом в лекції (в частині про FeedForward мережі).

  У нас модель складається з лінійної комбінації вхідних значень і застосування фукнції сигмоїда. Тож, нейромережа буде складатись з лінійного шару `nn.Linear` і використання активації `nn.Sigmid`. У створеному класі мають бути реалізовані методи `__init__` з ініціалізацією шарів і метод `forward` для виконання прямого проходу моделі через лінійний шар і функцію активації.

  Створіть екземпляр класу `LogReg` в змінній `model`.

In [48]:
import torch.nn as nn

class LogReg(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)
    self.act = nn.Sigmoid()
  def forward(self, x):
    x = self.linear(x)
    x = self.act(x)
    return x

model = LogReg()
print(model)


LogReg(
  (linear): Linear(in_features=3, out_features=1, bias=True)
  (act): Sigmoid()
)


10. Задайте оптимізатор `Stockastic Gradient Descent` в змінній `opt` для навчання моделі логістичної регресії. А також визначіть в змінній `loss` функцію втрат `binary_cross_entropy` з модуля `torch.nn.functional` для обчислення втрат моделі. Обчисліть втрати для поточних передбачень і міток, а потім виведіть їх. Зробіть висновок, чи моделі вдалось навчитись?

In [49]:
import torch.nn.functional as F

opt = torch.optim.SGD(model.parameters(), 1e-3)
loss_fn = F.binary_cross_entropy

11. Візьміть з лекції функцію для тренування моделі з відстеженням значень втрат і навчіть щойно визначену модель на 1000 епохах. Виведіть після цього графік зміни loss, фінальні передбачення і значення таргетів.

In [50]:
# Модифікована функцію fit для відстеження втрат
def fit_return_loss(num_epochs, model, loss_fn, opt, train_dl):
    losses = []
    for epoch in range(num_epochs):
        # Ініціалізуємо акумулятор для втрат
        total_loss = 0

        for xb, yb in train_dl:
            # Генеруємо передбачення
            pred = model(xb)

            # Обчислюємо втрати
            loss = loss_fn(pred, yb)

            # Виконуємо градієнтний спуск
            loss.backward()
            opt.step()
            opt.zero_grad()

            # Накопичуємо втрати
            total_loss += loss.item()

        # Обчислюємо середні втрати для епохи
        avg_loss = total_loss / len(train_dl)
        losses.append(avg_loss)

        # Виводимо підсумок епохи
        if (epoch + 1) % 10 == 0:
          print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')
    return losses


losses = fit_return_loss(1000, model, loss_fn, opt, train_dl)

preds = model(inputs)
print(preds)
print(targets)

Epoch [10/1000], Loss: 0.6604
Epoch [20/1000], Loss: 0.2981
Epoch [30/1000], Loss: 0.2335
Epoch [40/1000], Loss: 0.1327
Epoch [50/1000], Loss: 0.1072
Epoch [60/1000], Loss: 0.0964
Epoch [70/1000], Loss: 0.0964
Epoch [80/1000], Loss: 0.1214
Epoch [90/1000], Loss: 0.2128
Epoch [100/1000], Loss: 0.0790
Epoch [110/1000], Loss: 0.0798
Epoch [120/1000], Loss: 0.0791
Epoch [130/1000], Loss: 0.0813
Epoch [140/1000], Loss: 0.0706
Epoch [150/1000], Loss: 0.0757
Epoch [160/1000], Loss: 0.0658
Epoch [170/1000], Loss: 0.0672
Epoch [180/1000], Loss: 0.0618
Epoch [190/1000], Loss: 0.0848
Epoch [200/1000], Loss: 0.0909
Epoch [210/1000], Loss: 0.0806
Epoch [220/1000], Loss: 0.0577
Epoch [230/1000], Loss: 0.0577
Epoch [240/1000], Loss: 0.0549
Epoch [250/1000], Loss: 0.0634
Epoch [260/1000], Loss: 0.0683
Epoch [270/1000], Loss: 0.0742
Epoch [280/1000], Loss: 0.0548
Epoch [290/1000], Loss: 0.0488
Epoch [300/1000], Loss: 0.0494
Epoch [310/1000], Loss: 0.0440
Epoch [320/1000], Loss: 0.0467
Epoch [330/1000],

In [51]:
preds = preds > 0.5
preds = preds.float()

print(preds)
print(targets)

tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]])
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.]])
